# Convert a spreadsheet to a website.

A couple of rules:

Name your pages like ```App layout``` or ```Shop layout```, with an optional ```Shop data``` which will automatically load as a data table to be used in the template like ```{{#shop-data}}{{description}}{{/shop-data}}```. This would repeat for every row in the data table printing out all of the description columns. Data tables should list the column names as the first row. Hiding sheets doesn't matter to the converter but makes it easier to manage lots of pages or see only the one's you are working on.

Use HTML or Mustache or Markdown in your sheets. Rows are automatically converted to top level ```<div>``` elements. The class ```col-2``` is added to count the number of columns. Cells are automatically converted to ```<div>``` as children on the row ```<div>```. The class ```cell-0``` is automatically added as the index of the cell, not including any variables in between.

Beginning a cell with ":" comma creates variables and the cell right of a variable is used at the value. Variables do not affect column count or HTML output. Variables are combined with subpages, that is, any variable set on one page can be used page another page loaded by a URL, ```{{> section}}```, or ```::render```.

Special variables are:

```:logo```, which shows up in the favicon.

```:title``` is used as page title.

```::render``` forces another template to render whether or not there is a link to it, this is a list and can be used as many times as you want.

```{{> section}}``` inserts a {{#section}}{{/section}} mustache template, combines section from mustach and render from this framework. Pages that are not linked to, rendered, or sectioned, won't be scanned at all. Sections should be refered to in lowercase with spaces converted to dashes and without the word ```layout``` attached, i.e. ```{{> shop-menu}}```.

**Now for the complicated part!** ```/groups/1``` will load the layout ```groups``` and filter by the ```url``` or ```link``` column to match a single group. If no ```link``` or ```url``` column matches, it will search the column named after the page, the ```groups``` column will be used to match data categorically, all data rows matching ```/group/1``` will be available for repetion with a ```{{#groups-data}}{{/groups-data}}``` section tag.

The same rule applies to section includes, but even better! You can include a filtered section from the parents match, if you had groups and subgroups for each of the groups. ```{{> subgroup/groups}}``` will use the ```{{groups}}``` variable from the current page, and filter subgroups accordingly. Any subgroup with the subgroup matching ```/group/1``` will be available for repetition using ```{{#subgroup/groups-data}}{{/subgroup/groups-data}}```.

URLs such as ```/link``` inherently forces the spreadsheet page named Link layout, or Link data to render. Links to other pages are automatically detected. Names of pages are automatically converted to lowercase, with dashes instead of spaces.

Use ```::variable``` to create a list such as ```::stylesheet``` or ```:scripts```.

[Markdown description at markdownguide.org](https://www.markdownguide.org/basic-syntax)

[Mustache guide from NodeJs](https://github.com/janl/mustache.js)


TODO:

Inlining all styles and scripts and images as data-uri for faster load times.

Automatically detecting which features to add such as ```Buy Now``` buttons and contact us chats.

readme.md?


In [ ]:
// placeholder for readme

filter data sheet based on url?

In [ ]:
var TRIM = /^\s*\/\s*|\s*\/\s*$/ig;

var compareLink = (dataValue, base, link) => {
    // TODO: match more?
    return (dataValue || '').replace(TRIM, '') === link
        || (dataValue || '').replace(TRIM, '') === base + '/' + link
        || dataValue === link.split('/').slice(1).join('/')
        || dataValue === base
}

function filteredData(value, properties, categorical) {
    return function(val, render) {
        var segments = value.split('/');
        var key = segments[0];
        var match = segments[1];
        var link = (render(`{{{${match}}}}`) || properties[match] || '').replace(TRIM, '');
        var base = (render(`{{{base}}}`) || properties['base'] || '').replace(TRIM, '');
        if(link.substr(0, base.length) === base) {
            link = link.substr(base.length).replace(TRIM, '');
        }
        var matchKey = key + '-filtered-data';
        var restore = properties[key + '-original-data'];
        
        if(categorical) {
            // handle multiple results for use with categories
            properties[matchKey] = restore.filter(data => compareLink(data[key], base, link))
        } else {
            // TODO: use the third argument to find unique match in the data set
            properties[matchKey] = [restore.filter(data => compareLink(data['link'], base, link)
                                                   || compareLink(data['url'], base, link))[0]];
            if(typeof properties[matchKey][0] === 'undefined') {
                throw new Error(`Unique key not found: ${key} ${match} ${link}`)
            }
        }
        
        console.log(`Rendering filtered ${categorical ? 'categorical' : 'unique'}: ${key} ${match} ${link} ${properties[matchKey].length}`);
        var rendered = render(`{{#${matchKey}}}${val}{{/${matchKey}}}`);
        delete properties[matchKey];
        return rendered;
    }
}

module.exports = filteredData;


google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');
var filteredData = importer.import('filter data sheet based on url');

function getTemplateProperties(key, properties, templates) {
    var promise;
    if(templates[key].data) {
        if(typeof templates[key].data.rows != 'undefined') {
            promise = Promise.resolve(templates[key].data.rows)
        } else {
            promise = getDataSheet(templates[key].data)
                .then(data => (templates[key].data.rows = data))
        }
    } else {
        promise = Promise.resolve();
    }
    
    return promise
        .then(data => (properties[key + '-data'] = data))
        .then(() => getSections(key, properties, templates))
}

function matchSections(cell, properties, templates) {
    return importer.regexToArray(/\{\{>\s*(.*?)\}\}/ig, cell, 1).reduce((promise, value) => {
        var segments = value.split('/');
        var key = segments[0];

        if(typeof templates[key] != 'undefined') {
            console.log(`Reading partial: ${key} from ${value}`);
            return promise
                .then(() => getTemplateProperties(key, properties, templates))
                .then(() => {
                    // add a special partial for the filtered data
                    if(segments.length > 1) {
                        properties[value] = properties[key];
                        if(typeof properties[key + '-original-data'] == 'undefined'
                          && typeof properties[key + '-data'] != 'function') {
                            properties[key + '-original-data'] = properties[key + '-data'];
                        }
                        var categorical = (properties[key + '-original-data'] || [])
                            .filter(row => row.hasOwnProperty(key)).length > 0;
                        // automatically wrap unique templates in a data section for accessing filtered properties
                        if(!categorical && !properties[key].includes(`{{#${key}-data}}`)
                            && typeof properties[key + '-original-data'] !== 'undefined') {
                            properties[value] = `{{#${key}-data}}${properties[key]}{{/${key}-data}}`;
                        }
                        // run the filtered function instead of using the array
                        properties[key + '-data'] = filteredData.bind(
                            null, value, properties, categorical);
                    }
                })
        } else {
            console.log(`Section ${key} not found!`);
            return promise;
        }
    }, Promise.resolve())
}

// must do this up front so we can process all data
function getSections(key, properties, templates) {
    var promise;
    if(templates[key].template) {
        if(typeof templates[key].template.rows != 'undefined') {
            promise = Promise.resolve(templates[key].template.rows);
        } else {
            promise = getRows(templates[key].template)
                .then(rows => (templates[key].template.rows = rows))
        }
    } else {
        promise = Promise.resolve();
    }
    
    var rows;
    return promise
        .then(rs => {
            rows = rs || [];
            return rows.reduce((promise, row, i) => {
                return row.reduce((promise, c, j) => {
                    return promise.then(() => matchSections(c, properties, templates));
                }, promise);
            }, Promise.resolve());
        })
        // detect if this is just a wrapper template and don't render rows, do a direct replacement
        .then(() => rows.length === 1 && rows[0].length === 1
              && rows[0][0].match(/\{\{> (.*?)\/(.*?)-(.*?)-link\}\}/ig)
              ? rows[0][0]
              : renderRows(key, rows, properties, templates))
        .then(template => (properties[key] = template));
}

module.exports = getTemplateProperties;


google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var importer = require('../Core');
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40);
}

function escape(s) {
    return s.replace(/[-\/\\^$*+?.()|[\]{}]/g, '\\$&');
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    if(typeof data != 'object') {
        return [];
    }
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    
    return dataKeys
        .filter(k => c.match(new RegExp('\\{\\{\\s*[>#\\/]?\\s*' + escape(k) + '\\s*\\}\\}', 'ig')))
}

function renderRows(layoutTitle, rows, properties, templates) {
    if(!rows) {
        rows = [];
    }
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            var dataClasses = getDataClasses(c, typeof properties[layoutTitle + '-data'] == 'function'
                                            ? properties[layoutTitle + '-original-data']
                                            : properties[layoutTitle + '-data'])
                .map(k => 'val-' + safeName(k))
                .join(' ')
            var sectionClasses = getDataClasses(c, [properties, templates])
                .map(k => 'section-' + safeName(k))
                .join(' ')
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                var value = row[j + 1];
                if (c.substr(0, 2) === '::' || c === ':render') {
                    if(c === ':render') c = '::render'; // just to fix using it below
                    if(typeof properties[c.substr(2)] == 'undefined') {
                        properties[c.substr(2)] = [];
                    }
                    properties[c.substr(2)][properties[c.substr(2)].length] = value;
                } else {
                    properties[c.substr(1)] = value;
                }
            // render markdown content if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1] && row[j - 1].substr(0, 1) !== ':') {
                var mdHtml = md.render(c).replace(/\{\{\s*&gt;\s*/ig, '{{> ');
                // if all markdown did was insert a paragraph and line break, use value instead
                if(mdHtml.replace(/<\/?p>|[\s\n\r]+/ig, '').trim() == c.replace(/<\/?p>|[\s\n\r]+/ig, '').trim()) {
                    mdHtml = c;
                }

                arr[arr.length] = `
<div class="cell-${arr.length} ${dataClasses} ${sectionClasses}">
${mdHtml}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            arr[arr.length] = `
<div class="row-${arr.length} ${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;



output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function safeName(val, render) {
    return render(val).replace(/[^a-z0-9\-]/ig, '-').substr(0, 40)
}

function toJSON(val, render) {
    return render(JSON.stringify(val))
}

function wrapTemplate(path, html, properties) {
    properties['safeName'] = () => safeName;
    properties['toJSON'] = () => toJSON;

    var stylesheet = '', script = '', banner = '', domain = '', base = '';
    if(path.includes('packs')) {
        debugger;
    }
    
    // turn this in to an array/section
    if(typeof properties['stylesheet'] != 'undefined') {
        if(typeof properties['stylesheet'] === 'string') {
            properties['stylesheet'] = [properties['stylesheet']];
        }
        stylesheet = '{{#stylesheet}}<link rel="stylesheet" media="screen" href="{{.}}">{{/stylesheet}}';
    }
    if(typeof properties['script'] != 'undefined') {
        if(typeof properties['script'] === 'string') {
            properties['script'] = [properties['script']];
        }
        script = '{{#script}}<script src="{{.}}"></script>{{/script}}';
    }
    if(typeof properties['banner'] != 'undefined') {
        banner = 'background-image: url({{{banner}}});';
    }
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '{{domain}}' : 'https://{{domain}}';
    }
    if(typeof properties['base'] != 'undefined') {
        base = '<base href="/{{base}}" />';
    }
    
    // automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
${stylesheet}
${base}
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${path}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
${banner}
}
</style>
</head>
<body class="${path.replace(/\//ig, ' ')} {{class}}">
${html}
${script}
</body>
</html>`;
    Mustache.parse(pageHtml);
    // use properties for view and for partials
    return Mustache.render(pageHtml, properties, properties);
}

module.exports = wrapTemplate;



find known routes to sheets?


In [ ]:
var Mustache = require('mustache');
var importer = require('../Core');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');

// combine with "getSections" by using fake "{{> url/path}}" include
function collectRoutes(routes, properties, templates, rendered) {
    var local = routes.concat(properties['render'] || [])
        // protocol means it's absolute remote path and not to try to generate it
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':'))
        .filter(link => !rendered.includes(link))

    local.forEach(link => rendered[rendered.length] = link)
    
    return importer.runAllPromises(local
        // promise in series so there is no data collisions
        .map(link => resolve => {
            link = link.replace(/^\/|\/$/ig, '');
            if(link === '') {
                link = Object.keys(templates).filter(t => templates[t].template
                                                     && templates[t].template.properties
                                                     && templates[t].template.properties.index == 0)[0];
            }
            var trimmedBase = (properties['base'] || '').replace(/^\/|\/$/ig, '');            
            if(link.substr(0, trimmedBase.length) === trimmedBase) {
                link = link.substr(trimmedBase.length).replace(/^\/|\/$/ig, '');
            }
            // any part of a path can contain the reference to a page template
            var key = link.split('/')
                .filter(segment => templates[segment] && templates[segment].template)[0];
            var newProps = Object.assign({}, properties);
        
            // create a temporary template to filter by
            newProps[key + '-' + key + '-link'] = link;
            templates[key + '-' + key] = {template: {rows: [[`{{> ${key}/${key}-${key}-link}}`]]}}
        
            return getTemplateProperties(key + '-' + key, newProps, templates)
                .then(() => wrapTemplate(link, newProps[key + '-' + key], newProps))
                .then(page => {
                    var pages = {};
                    pages[link] = page;
                    resolve(pages)
                })
        }))
        .then(results => results.reduce((obj, r) => Object.assign(obj, r), {}))
}

module.exports = collectRoutes;



collect external content and resources?


In [ ]:
var {Readable} = require('stream');
var {JSDOM, XPathResult} = require('jsdom');
var importer = require('../Core');
var getArrayXPath = importer.import('get xpath array');

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40);
}

function collectExternalResources(page, rendered, routes) {
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    // TODO: add IDs to h1, h2, h3, etc elements that match their text contents
    var headingsObjs = getArrayXPath('(//h1|//h2|//h3|//h4)[not(@id)]', dom.window.document.body);
    headingsObjs.forEach(h => h.setAttribute('id', safeName(h.textContent)));
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    // TODO: convert images and add timestamps, add svg
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    // TODO: scan for urls and inline
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    // TODO: add timestamps and inline
    var scriptsObjs = getArrayXPath('//script[@src]', dom.window.document.body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    // TODO: add CSS imports
    var backgrounds = importer.regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var searches = imgs.concat(styles).concat(backgrounds).concat(scripts)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && rendered.indexOf(cur) === -1);
    
    links.forEach(s => routes[routes.length] = s)
    searches.forEach(s => rendered[rendered.length] = s)
    
    // TODO: copy resource images to output directory
    var newPage = dom.window.document.documentElement.outerHTML
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

module.exports = collectExternalResources;


collect google sheets resources?


In [ ]:
var importer = require('../Core');
var streamToGoogle = importer.import('upload files google cloud');
var copyFileBucket = importer.import('copy file bucket storage');
var collectExternalResources = importer.import('collect external content and resources');
var collectRoutes = importer.import('find known routes to sheets');

var timestamp = (new Date()).getTime();

// detect links and write out every part of the site
function collectTemplateResources(path, page, properties, templates, bucketName, rendered, sTG) {
    if(!rendered) {
        rendered = [];
    }
    
    // if it is the first page in the template, rename it to index.html
    var template = path.split('/')
        .filter(segment => templates[segment] && templates[segment].template)[0];
    if(template && templates[template].template.properties.index === 0) {
        console.log(`Using ${path} as index.html`);
        path = 'index';
    }
    var trimmedBase = (properties['base'] || '').replace(/^\/|\/$/ig, '');
    if(path.substr(0, trimmedBase.length) !== trimmedBase) {
        path = trimmedBase + '/' + path;
    }

    // TODO: add timestamps to generated content
    
    var routes = [];
    return collectExternalResources(page, rendered, routes)
        .then(stream => (console.log(`Emitting ${path}`), stream))
        .then(stream => (sTG ? sTG : streamToGoogle)(path + '.html', bucketName, stream, {
            contentType: 'text/html; charset=utf-8'
        } /* TODO: insert permission settings for user directory */))
        .then(() => !sTG ? copyFileBucket(bucketName, path + '.html') : Promise.resolve(false))
        .then(() => collectRoutes(routes, properties, templates, rendered))
        .then(pages => importer.runAllPromises(Object.keys(pages).map(fileName => resolve => 
            collectTemplateResources(fileName, pages[fileName], properties, templates, bucketName, rendered, sTG)
                .then(() => resolve()))))
        .then(() => rendered)
}

module.exports = collectTemplateResources;


test google sheets resources?


In [ ]:
var assert = require('assert');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var findSimilarFile = importer.import('find similar file');

var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));

// locally based utility for editing styles
function streamToOutput(fileUrl, bucketName, stream) {    
     return new Promise((resolve, reject) => {
        if(typeof stream == 'object') {
            var outputPath = path.join(path.resolve('./.output/'), fileUrl.replace(/\?.*/ig, ''));
            if(!fs.existsSync(path.dirname(outputPath))) {
                fs.mkdirSync(path.dirname(outputPath));
            }
            var writeStream = fs.createWriteStream(outputPath);
            stream.pipe(writeStream)
            .on("error", (err) => {
                reject(err);
            })
            .on('finish', () => {
                resolve(outputPath);
            });
        } else {
            var outputPath = path.join(path.resolve('./.output/'), path.basename(fileUrl.replace(/\?.*/ig, '')));
            var writeStream = fs.createWriteStream(outputPath);
            (fileUrl.includes('https://') ? https : http).get(fileUrl, response => {
                response.pipe(writeStream)
                .on("error", (err) => {
                    reject(err);
                })
                .on('finish', () => {
                    resolve(outputPath);
                });
            }).catch(e => console.log(e));
        }
     });
}

function outputTest(page, bucketName, key, templates, properties) {
    return collectTemplateResources(key, page, properties, templates, bucketName, false, streamToOutput)
        .then(resources => {
            console.log(resources);
            return Promise.all(resources.filter(r => !r.includes(':') && r.includes('.'))
                .map(r => {
                    // TODO: search notebooks for code so library can be saved to git
                    return findSimilarFile(path.basename(r), DOWNLOAD_DIR)
                        .then(f => {
                            if(f) {
                                return streamToOutput(
                                    f,
                                    bucketName,
                                    fs.createReadStream(path.join(DOWNLOAD_DIR, f)));
                            } else {
                                return Promise.resolve(false);
                            }
                        });
                }));
        });
}

function importTest(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var templates, key, properties = {};
    
    return getTemplates(docId)
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => {
            assert(page.length > 0, 'should have a page');
            return outputTest(page, domain, key, templates, properties);
        })
}

if(typeof describe !== 'undefined') {
    describe('output template resources', () => {

        it('should process at least one template', () => {
            var docsId = '1QeZ3WduNFmjtNf_Q3Xe5zAh8dzP_nn7nr-AQU0_hXg8';
            var bucketName = 'sheet-to-web.com';
            return importTest(docsId, bucketName);
        }).timeout(60000)
    })
    
}

module.exports = importTest;


google sheet handler?

get sheet identifier from link?


In [ ]:
var util = require('util');
var uuid = require('uuid/v1');
var getInfo = importer.import('get google sheet info');
var getDataSheet = importer.import('google sheet array objects');
var addRow = importer.import('add row data google sheet');
var updateRow = importer.import('update a row in google sheets')

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var project = 'spahaha-ea443';

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40).toLowerCase();
}

function getSheet(link, email) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name, title;
    
    return getInfo(link)
        // return assigned subdomain
        .then(info => {
            name = safeName(info.properties.title.replace(/\s*(configuration|config)\s*/ig, ''))
                + '-' + uuid().substr(0, 5);
            title = info.properties.title;
            return getDataSheet(purchaseId, 'Purchases');
        })
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(match) {
                console.log(`Purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
                // update name and email
                return updateRow(purchaseId, r => r.sheet == docId, {
                    name: title,
                    email: email
                })
                    .then(() => match.domain || match.bucket);
            }
        
            console.log(`Adding product row for ${docId} ${name}`)
            return addRow(purchaseId, {
                timestamp: Date.now(),
                name: title,
                email: email || '',
                address: '',
                domain: '',
                bucket: name + '.sheet-to-web.com',
                project: project,
                sheet: docId
            })
                .then(() => name + '.sheet-to-web.com')
        })
}

module.exports = getSheet;



sheet marketing import handler?

sheet marketing import?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

function importSheet(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var properties = {}, templates, key;
    
    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain != '' && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            return getTemplates(docId)
        })
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
        })
        .then(() => wrapTemplate(key, properties[key], properties))
        .then(page => collectTemplateResources(key, page, properties, templates, domain || match.bucket))
        .then(resources => {
            console.log(resources);
            return resources;
        })
}

module.exports = importSheet;



sheet backend handler?

setup sheet backend?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var project;

    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain != '' && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            project = match.project;
            domain = domain || match.bucket;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
        .then(() => domain)
}

module.exports = setupBackend;



create a sheet handler?

create a copy of marketing template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyMarketing(email) {
    var fileId;
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Marketing site')[0].id)
        .then(() => copyFile(fileId, 'Marketing site ' + uuid().substr(0, 5)))
        .then(id => insertPermission(id, email))
        .then(() => getSheet(fileId, email))
        .then(() => fileId)
}

module.exports = copyMarketing;



package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
